In [7]:
import pandas as pd
import geopandas as gp
import numpy as np
import re

# 连接房地产信息和坐标

In [8]:
community_info = pd.read_json("../community_info.jsonl", lines=True)
community_info = community_info.join(pd.json_normalize(community_info["info"]))
community_info = community_info.loc[community_info["info"] != {}, :]
community_info = community_info.drop(columns="info")
community_info = community_info.replace({
    np.nan: pd.NA
})
community_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4794 entries, 0 to 4794
Data columns (total 77 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      4794 non-null   object
 1   link      4794 non-null   object
 2   district  4794 non-null   object
 3   物业类别      617 non-null    object
 4   项目特色      756 non-null    object
 5   建筑类别      617 non-null    object
 6   装修状况      756 non-null    object
 7   产权年限      617 non-null    object
 8   环线位置      4443 non-null   object
 9   开发商       4794 non-null   object
 10  楼盘地址      617 non-null    object
 11  销售状态      617 non-null    object
 12  开盘时间      617 non-null    object
 13  交房时间      617 non-null    object
 14  售楼地址      1600 non-null   object
 15  咨询电话      617 non-null    object
 16  主力户型      617 non-null    object
 17  预售许可证     617 non-null    object
 18  占地面积      4793 non-null   object
 19  建筑面积      4794 non-null   object
 20  容积率       4794 non-null   object
 21  绿化率       4794 non-

In [9]:
community_id = pd.read_csv("../community.csv", index_col=0)
community_id = pd.concat([community_id, community_info.drop(columns="name")], axis=1)
community_id.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4794 entries, 0 to 4794
Data columns (total 79 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uuid      4794 non-null   object
 1   name      4794 non-null   object
 2   address   4794 non-null   object
 3   link      4794 non-null   object
 4   district  4794 non-null   object
 5   物业类别      617 non-null    object
 6   项目特色      756 non-null    object
 7   建筑类别      617 non-null    object
 8   装修状况      756 non-null    object
 9   产权年限      617 non-null    object
 10  环线位置      4443 non-null   object
 11  开发商       4794 non-null   object
 12  楼盘地址      617 non-null    object
 13  销售状态      617 non-null    object
 14  开盘时间      617 non-null    object
 15  交房时间      617 non-null    object
 16  售楼地址      1600 non-null   object
 17  咨询电话      617 non-null    object
 18  主力户型      617 non-null    object
 19  预售许可证     617 non-null    object
 20  占地面积      4793 non-null   object
 21  建筑面积      4794 non-

In [10]:
community_loc = pd.read_json("../community_geolocation.jsonl", lines=True).drop(columns=["index", "name", "address", "waiting"])
community_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4300 entries, 0 to 4299
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   uuid    4300 non-null   object 
 1   lon     4300 non-null   float64
 2   lat     4300 non-null   float64
dtypes: float64(2), object(1)
memory usage: 100.9+ KB


In [11]:
community = pd.merge(community_id, community_loc, how="inner", on="uuid")
community.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4300 entries, 0 to 4299
Data columns (total 81 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   uuid      4300 non-null   object 
 1   name      4300 non-null   object 
 2   address   4300 non-null   object 
 3   link      4300 non-null   object 
 4   district  4300 non-null   object 
 5   物业类别      130 non-null    object 
 6   项目特色      287 non-null    object 
 7   建筑类别      130 non-null    object 
 8   装修状况      287 non-null    object 
 9   产权年限      130 non-null    object 
 10  环线位置      4214 non-null   object 
 11  开发商       4300 non-null   object 
 12  楼盘地址      130 non-null    object 
 13  销售状态      130 non-null    object 
 14  开盘时间      130 non-null    object 
 15  交房时间      130 non-null    object 
 16  售楼地址      1205 non-null   object 
 17  咨询电话      130 non-null    object 
 18  主力户型      130 non-null    object 
 19  预售许可证     130 non-null    object 
 20  占地面积      4299 non-null   obje

# 筛选需要的变量

In [13]:
community_park = community[['name', 'district', '停车位', '停车位2', '停车位描述', '建筑类型', '建筑类别', '总户数', '房屋总数', 'lon', 'lat']]
community_park['building_type'] = community_park.apply(lambda x: x.建筑类型 if pd.isna(x.建筑类别) else x.建筑类别, axis=1)
community_park = community_park.map(lambda x: pd.NA if type(x) == str and re.match("暂无|待定", x) != None else x)
community_park['carpark_describe'] = community_park.apply(lambda r: ";".join([re.subn(r'\s', '', x)[0] for x in [r.停车位, r.停车位2, r.停车位描述] if x is not pd.NA]), axis=1)
community_park = community_park.drop(columns=['停车位', '停车位2', '停车位描述', '建筑类型', '建筑类别'])
community_park

/var/folders/_t/clbb14dx1731tq66wy5kzct80000gn/T/ipykernel_87218/3634340814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  community_park['building_type'] = community_park.apply(lambda x: x.建筑类型 if pd.isna(x.建筑类别) else x.建筑类别, axis=1)


,name,district,总户数,房屋总数,lon,lat,building_type,carpark_describe
0,信达天樾九章,金水区,2253户,<NA>,113.647763,34.842079,板楼,车位配比1：1.14
1,保利缦城和颂,金水区,<NA>,<NA>,113.746073,34.824377,板楼,3529个
2,中豫·湖城印象,金水区,1651户,<NA>,113.709333,34.850858,板楼,共1753个
3,美盛教育港湾,金水区,3678户,<NA>,113.679263,34.818176,板塔结合,停车位共3782个
4,绿城·沐春歌,金水区,943户,<NA>,113.666300,34.837099,板楼,970个;共970个
...,...,...,...,...,...,...,...,...
4295,郑州洺悦融园,管城区,1435户,<NA>,113.696920,34.717059,板楼,1540个;机动车位总数1540个；其中地上车位数160个，地下车位数1380个
4296,富田城·九鼎华府,管城区,4021户,<NA>,113.747677,34.668123,板楼,共722个停车位；车位配比约1：1
4297,商都阜园,管城区,2664户,<NA>,113.629707,34.700897,板楼,3005个;3005
4298,东亚 · 京尊,管城区,<NA>,<NA>,113.670280,34.702791,塔楼,规划停车位814个


In [14]:
def parse_properties(t):
    prop_desc = t.总户数 if not pd.isna(t.总户数) else t.房屋总数
    if not pd.isna(prop_desc):
        prop_num_str = re.search(r"\d+", prop_desc)
        if prop_num_str is not None:
            return int(prop_num_str.group())
    return None
community_park['properties'] = community_park.apply(parse_properties, axis=1).tolist()
community_park = community_park.drop(columns=['总户数', '房屋总数'])
community_park

,name,district,lon,lat,building_type,carpark_describe,properties
0,信达天樾九章,金水区,113.647763,34.842079,板楼,车位配比1：1.14,2253.0
1,保利缦城和颂,金水区,113.746073,34.824377,板楼,3529个,NaN
2,中豫·湖城印象,金水区,113.709333,34.850858,板楼,共1753个,1651.0
3,美盛教育港湾,金水区,113.679263,34.818176,板塔结合,停车位共3782个,3678.0
4,绿城·沐春歌,金水区,113.666300,34.837099,板楼,970个;共970个,943.0
...,...,...,...,...,...,...,...
4295,郑州洺悦融园,管城区,113.696920,34.717059,板楼,1540个;机动车位总数1540个；其中地上车位数160个，地下车位数1380个,1435.0
4296,富田城·九鼎华府,管城区,113.747677,34.668123,板楼,共722个停车位；车位配比约1：1,4021.0
4297,商都阜园,管城区,113.629707,34.700897,板楼,3005个;3005,2664.0
4298,东亚 · 京尊,管城区,113.670280,34.702791,塔楼,规划停车位814个,NaN


In [15]:
def parse_parking(t):
    under_num, upper_num, total_num, ratio = (None, None, None, None)
    upper_regex = re.compile(r"((地上((停)?车位(数|共约?|为)?)?)(：|:)?(\d+)(元)?)|((\d+)((个|左右)?地上))")
    upper_desc = re.findall(upper_regex, t)
    if len(upper_desc) > 0:
        upper_groups = [x for x in upper_desc[-1] if x != ""]
        if upper_groups[-1] != '元':
            upper_num = int([x for x in upper_groups if re.match(r"^\d+$", x) is not None][-1])
    t_reduce = re.subn(upper_regex, "", t)[0]
    under_desc = re.findall(r"((地下((停)?车位(数|共约?|为)?)?)(：|:)?(\d+)(元)?)|((\d+)((个|左右)?地下))", t_reduce)
    if len(under_desc) > 0:
        under_groups = [x for x in under_desc[-1] if x != ""]
        if under_groups[-1] != '元':
            under_num = int([x for x in under_groups if re.match(r"^\d+$", x) is not None][-1])
    total_desc = re.findall(r"(((?<!非机动停车位)(共(有|设)?|(总数：)|约)(\d+))|(小区(共)?有(\d+))|((规划)(机动车|有)?(\d+))|(^(\d{1,})(.)?)|(;(\d+)(.)?)|((?<!非)(机动车停?车?(位|数量?|位数量?)(：|:)?)(\d+))|((?<![上下])停?车位数?(\d+))|((?<![上下])(\d+)个停?车位))(?!(\:|：))", t)
    if len(total_desc) > 0:
        total_groups = [x for x in total_desc[-1] if x != ""]
        if total_groups[-1] != '-' and total_groups[-1] != '.':
            total_nums = [x for x in total_groups if re.match(r"^\d+$", x) is not None]
            if len(total_nums) > 0:
                total_num = int(total_nums[0])
                if re.search(r"全地下停车位", t) != None:
                    under_num = total_num
                elif re.search(r"全地上停车位", t) != None:
                    upper_num = total_num
                sum_under_upper = (under_num or 0) + (upper_num or 0)
                if total_num < sum_under_upper:
                    total_num = sum_under_upper
            else:
                print("no total:", total_groups)
    if total_num is None:
        if under_num is not None or upper_num is not None:
            total_num = (under_num or 0) + (upper_num or 0)
    ratio_desc = re.search(r"(\d+)(：|:)(\d+(.\d+)?)", t)
    if ratio_desc != None:
        ratio = re.subn("：", ":", ratio_desc.group())[0]
    return [upper_num, under_num, total_num, ratio]

# [x for x in community_park['carpark_describe'].apply(parse_parking).tolist() if not np.isnan(x)]
community_park[['park_upper', 'park_under', 'park_total', 'park_ratio']] = community_park['carpark_describe'].apply(parse_parking).tolist()

In [16]:
community_park.to_csv("community_park.csv")

# 将房地产转换为空间矢量数据

In [17]:
### 首先需要将高德坐标系转换为 WGS84 坐标系
from coords_trans import CoordinateTransform

In [18]:
transer = CoordinateTransform()

In [5]:
# community_park = pd.read_csv("./community_park.csv")

In [19]:
community_park[["lon_wgs", "lat_wgs"]] = [transer.gcj02_to_wgs84(u, v) for u, v in zip(community_park["lon"], community_park["lat"])]
community_park.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4300 entries, 0 to 4299
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              4300 non-null   object 
 1   district          4300 non-null   object 
 2   lon               4300 non-null   float64
 3   lat               4300 non-null   float64
 4   building_type     4087 non-null   object 
 5   carpark_describe  4300 non-null   object 
 6   properties        3946 non-null   float64
 7   park_upper        1610 non-null   object 
 8   park_under        417 non-null    object 
 9   park_total        2311 non-null   object 
 10  park_ratio        265 non-null    object 
 11  lon_wgs           4300 non-null   float64
 12  lat_wgs           4300 non-null   float64
dtypes: float64(5), object(8)
memory usage: 436.8+ KB


In [20]:
community_geo = gp.points_from_xy(community_park["lon_wgs"], community_park["lat_wgs"])
community_gdf = gp.GeoDataFrame(community_park, geometry=community_geo, crs=4326)
community_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4300 entries, 0 to 4299
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   name              4300 non-null   object  
 1   district          4300 non-null   object  
 2   lon               4300 non-null   float64 
 3   lat               4300 non-null   float64 
 4   building_type     4087 non-null   object  
 5   carpark_describe  4300 non-null   object  
 6   properties        3946 non-null   float64 
 7   park_upper        1610 non-null   object  
 8   park_under        417 non-null    object  
 9   park_total        2311 non-null   object  
 10  park_ratio        265 non-null    object  
 11  lon_wgs           4300 non-null   float64 
 12  lat_wgs           4300 non-null   float64 
 13  geometry          4300 non-null   geometry
dtypes: float64(5), geometry(1), object(8)
memory usage: 470.4+ KB


In [22]:
community_gdf.to_file("../shp/community_park.gpkg")

# 查找没有编码出坐标的小区

In [36]:
uuid_found = community_loc["uuid"].tolist()
community_noloc = community_id.loc[[(x not in uuid_found) for x in community_id["uuid"]], :]
community_noloc

,uuid,name,address
23,8de2570c-4739-45fe-bf49-affaae932dcb,绿地海珀兰轩,经开南三环与机场快速路交汇处
80,35871bae-b1eb-4e57-b1f7-cbb61274edde,河南省水利第二工程局家属院,经开
81,4df3f0a7-4c02-42dd-876e-954a1e29afcc,经济开发区公安分局家属院,经开
94,ee9d6c55-603d-4554-924e-4eb38c898898,经开管委会家属院,经开航海路三大街交汇处东南角
99,4349bc80-44ed-4f15-ba0a-2677ccedae45,航海路第二大街29号院,经开
...,...,...,...
4790,97abfe39-25dc-4b4f-b285-65690e68f7c1,瀚海·光辉城市,航空港区志洋路孙武路交会处东南
4791,610777e5-7ff7-4a77-b122-464c3d59724d,浩创梧桐花语,航空港区薛店神州路与枣园路交会处
4792,63110159-4347-4096-8665-ea89b6c8a3f6,中绘·集云筑,航空港区华夏大道与领航路交汇处
4793,00eb2a8b-4a9e-4097-8fe0-de19de519503,旭辉·空港时代,航空港区G107与志洋路交叉口东南角


In [37]:
community_noloc.to_csv("../community_noloc.csv")